## How to call functions with chat models

OpenAI has added a number of dazzling features in its AI push, one that sparkled for me was the ability to call external functions. I translate this as being able to integrate external APIs into openAI's fleet of APIs including the chat completions API (fact check) used by chatgpt, Open AIs flagship product. This integration invites a wide range of possibilties. In this article, I show one of those, very simple but useful, possibilities in my article below

This notebook covers how to use the Chat Completions API in combination with external functions to extend the capabilities of GPT models. In this case the chat completions api will be calling the weather api as an external function.

The external function in this case is a python function. The python function retrieves the current weather information for a specified location using the OpenWeather API. This function is then called in the openai function. 

This way you can ask chat completions to give you the temperature reading for any location like you would ask a local at said destination ;)

This is a very simple use case but knowing that you can integrate the OpenAI API with external APIs will give you the capability to build tools that can provide significant value to you in your day to day. From conversationally scheduling important events on your calendar (Google API) to making a last minute reservation to your favorite restuarant without having to go through multiple clicks.

Please sure your thoughts on how external functions can be integrated into OpenAI API!

## Prerequisites

## 1. OpenAI API key

Create OpenAI API key - https://help.openai.com/en/articles/4936850-where-do-i-find-my-api-key

# 2. Openweather API key

Create Openweather API key - https://openweathermap.org/appid#:~:text=Once%20you%20sign%20up%20using,additional%20API%20keys%20if%20needed.

# Running the function

In [ ]:
!pip install openai

### Import all the necessary modules needed. Most important are openai 

In [ ]:
import openai
import json
import os
import requests

##  Set the API keys environment

In [ ]:
os.environ["OPENAI_API_KEY"] = "<OPENAI_API_KEY>"
os.environ["OPEN_WEATHER_API_KEY"] = "<OPEN_WEATHER_API_KEY>"

## Assign a variable to the weather API and Open AI keys

In [ ]:
weather_api_key=os.getenv("OPEN_WEATHER_API_KEY")

In [ ]:
api_key=os.getenv("OPENAI_API_KEY")
openai.api_key = api_key

## Assign the OpenAI object to a variable

In [ ]:
client = openai.OpenAI()

### Function to convert from Kelvin to Fahrenheit

In [ ]:
def kelvin_to_fahrenheit(kelvin):
    return (kelvin - 273.15) * 9/5 + 32

### Define the get weather function that utilizes the Open weather API

In [ ]:
def get_weather(location,api_key=weather_api_key):
    """
    Get current weather information for a given location using OpenWeather API.

    Parameters:
    - api_key (str): Your OpenWeather API key
    - location (str): The location for which you want to get the weather information

    Returns:
    - dict: Weather information in a dictionary format
    """
    
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    
    location=location.lower()
    
    params = {
        'q': location,
        'appid': api_key,
    }

    try:
        response = requests.get(base_url, params=params)
        data = response.json()

        if response.status_code == 200:
            weather_info = {
                'temperature': kelvin_to_fahrenheit(data['main']['temp']),
            }
            return json.dumps(f"temperature_in_{location}: {weather_info['temperature']}")
        else:
            return {'error': f"Error {response.status_code}: {data['message']}"}

    except Exception as e:
        return {'error': f"An error occurred: {str(e)}"}

## Define the get temperature location function that utilizes the OpenAI API and integrates the external weather function

In [ ]:
def get_temperature_location(location):
    # Step 1: send the conversation and available functions to the model
    weather_prompt=f"What's the weather like in {location}? Can you only use the city name and specify temperature in Fahrenheit?"
    messages = [{"role": "user", "content": weather_prompt }]
    tools = [
        {
            "type": "function",
            "function": { #this is where the external function is specified
                "name": "get_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "api_key": {"type": "string", "description": "api keys"},
                    },
                    "required": ["location"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_weather": get_weather,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                location=function_args.get("location")
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response.choices[0].message.content

## Test out the function

In [ ]:
get_temperature_location("London")